In [1]:
# !pip install opencv-python
# !pip install tqdm

In [2]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from PIL import Image

In [3]:
path_tomos= '../data/pictures_raw/train'
path_csv = '../data/csv_raw/train_labels.csv'
path_32='/Volumes/DiskMorgane/bach_32/bach_1'

df = pd.read_csv(path_csv ).copy()
df

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
0,0,16,tomo_049310,-1.0,-1.0,-1.0,500,924,956,19.7,0
1,1,30,tomo_098751,-1.0,-1.0,-1.0,500,924,956,16.1,0
2,2,54,tomo_136c8d,-1.0,-1.0,-1.0,500,924,956,19.7,0
3,3,59,tomo_146de2,-1.0,-1.0,-1.0,500,924,956,16.1,0
4,4,84,tomo_1dc5f9,-1.0,-1.0,-1.0,500,924,956,19.7,0
...,...,...,...,...,...,...,...,...,...,...,...
732,732,99,tomo_226cd8,366.0,693.0,269.0,500,928,960,13.1,10
733,733,100,tomo_226cd8,386.0,693.0,319.0,500,928,960,13.1,10
734,734,101,tomo_226cd8,388.0,671.0,433.0,500,928,960,13.1,10
735,735,102,tomo_226cd8,411.0,318.0,740.0,500,928,960,13.1,10


In [4]:
# position du motor en 32 photos
df['motor_place'] = np.where( df['Motor_axis_0'] < 0,-1,(df['Motor_axis_0'] / df['Array_shape_axis_0'] * 32).astype(int))
# group by tomo_id
df = df.groupby('tomo_id').first().reset_index()
df

,tomo_id,Unnamed: 0,row_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors,motor_place
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,-1
1,tomo_00e047,338,1,169.0,546.0,603.0,300,959,928,15.6,1,18
2,tomo_00e463,709,2,235.0,403.0,137.0,500,924,956,19.7,6,15
3,tomo_01a877,404,8,147.0,638.0,286.0,300,960,928,13.1,1,15
4,tomo_02862f,339,9,101.0,351.0,120.0,300,959,928,15.6,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...
643,tomo_fe050c,402,732,138.0,134.0,542.0,300,959,928,15.6,1,14
644,tomo_fe85f6,155,733,-1.0,-1.0,-1.0,800,928,960,13.1,0,-1
645,tomo_fea6e8,264,734,-1.0,-1.0,-1.0,300,960,928,16.8,0,-1
646,tomo_ff505c,403,735,111.0,816.0,678.0,300,959,928,15.6,1,11


In [5]:

# put 1 in the line wher teh motor is , 0 if no

new_files = []

for _, file in df.iterrows():
    # Crear 32 copias de la fila
    for i in range(32):
        nueva_fila = file.copy()
        nueva_fila['motor_place'] = 1 if i == int(file['motor_place']) else 0
        new_files.append(nueva_fila)

df_expandido = pd.DataFrame(new_files)
# df_expandido.reset_index(drop=True, inplace=True)

df_expandido.head(50)


,tomo_id,Unnamed: 0,row_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors,motor_place
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0
0,tomo_003acc,283,0,-1.0,-1.0,-1.0,500,1912,1847,6.5,0,0


In [6]:

root_dir = '/Volumes/DiskMorgane/bach_32/bach_1'

# Filtrar solo carpetas que comienzan con 'tomo' y son directorios válidos
tomo_folders = sorted([
    f for f in os.listdir(root_dir)
    if f.startswith('tomo') and os.path.isdir(os.path.join(root_dir, f)) and not f.startswith("._")
])

# Lista para guardar todos los arrays
all_images = []

# Procesar cada carpeta
for tomo_folder in tqdm(tomo_folders, desc="Procesando carpetas tomo_*"):
    folder_path = os.path.join(root_dir, tomo_folder)

    # Filtrar imágenes válidas
    image_files = sorted([
        f for f in os.listdir(folder_path)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')) and not f.startswith("._")
    ])

    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️ No se pudo leer {img_path}")
            continue
        img_resized = cv2.resize(img, (512, 512))
        img_expanded = np.expand_dims(img_resized, axis=-1)  # (512, 512, 1)
        all_images.append(img_expanded)

# Convertir a array final
array_final = np.stack(all_images, axis=0)  # (N, 512, 512, 1)
print("✅ Forma final del array:", array_final.shape)


Procesando carpetas tomo_*: 100%|██████████| 648/648 [22:11<00:00,  2.05s/it]


✅ Forma final del array: (20736, 512, 512, 1)


In [7]:
# # usar para generar baches de 32 fotos por cada tomo
# def generate_mean_patches(
#     tomo_root_path,
#     csv_path,
#     patch_size=512,
#     patches_per_tomo=128,
#     num_tomos=None
# ):
#     """
#     Genera 128 imágenes 512x512 por tomo a partir del promedio de todas sus imágenes,
#     y devuelve los arrays junto con las etiquetas correspondientes.

#     Returns:
#         tuple: (X, y)
#             - X: np.ndarray shape (N, 512, 512, 1), valores normalizados
#             - y: np.ndarray shape (N,), etiquetas (Number_of_motors) replicadas por parche
#     """
#     all_patches = []
#     all_labels = []

#     # Cargar el DataFrame con la información
#     df = pd.read_csv(csv_path)

#     # Obtener lista de carpetas tipo tomo_*
#     tomo_dirs = sorted(glob(os.path.join(tomo_root_path, 'tomo_*')))
#     if num_tomos is not None:
#         tomo_dirs = tomo_dirs[:num_tomos]

#     for tomo_dir in tqdm(tomo_dirs, desc="Procesando tomos"):
#         tomo_id = os.path.basename(tomo_dir)

#         # Buscar la etiqueta correspondiente a este tomo
#         row = df[df['tomo_id'] == tomo_id]
#         if row.empty:
#             print(f"⚠️ No se encontró información en CSV para: {tomo_id}")
#             continue

#         # Etiqueta binaria: 1 si hay motores, 0 si no
#         num_motors = 1 if row['Number_of_motors'].values[0] > 0 else 0

#         # Cargar imágenes
#         image_paths = sorted(glob(os.path.join(tomo_dir, "*.png")) + glob(os.path.join(tomo_dir, "*.jpg")))
#         slices = []

#         for img_path in image_paths:
#             img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#             if img is not None:
#                 slices.append(img.astype(np.float32))

#         if not slices:
#             print(f"⚠️ Tomo vacío: {tomo_id}")
#             continue

#         # Promedio de slices → imagen 2D
#         volume = np.stack(slices, axis=0)
#         mean_image = np.mean(volume, axis=0)
#         mean_image = cv2.normalize(mean_image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#         # Extraer parches aleatorios 512x512
#         h_limit = mean_image.shape[0] - patch_size
#         w_limit = mean_image.shape[1] - patch_size

#         for _ in range(patches_per_tomo):
#             h = np.random.randint(0, h_limit + 1)
#             w = np.random.randint(0, w_limit + 1)
#             patch = mean_image[h:h+patch_size, w:w+patch_size]
#             all_patches.append(patch)
#             all_labels.append(num_motors)

#     X = np.array(all_patches)
#     y = np.array(all_labels)

#     # Agregar dimensión de canal y normalizar
#     X = np.expand_dims(X, axis=-1)
#     X = X.astype(np.float32) / 255.0

#     return X, y

# X, y= generate_mean_patches(
#     tomo_root_path = path_tomos,
#     csv_path = path_csv,
#     patch_size=512,
#     patches_per_tomo=32,
#     num_tomos=628
#     )

# def load_tomos_images_array(tomo_root_path, csv_path, num_images=32, image_size=512, num_tomos=None):
#     """
#     Carga las imágenes de cada tomo en un array con forma (num_tomos, 32, 512, 512, 1), redimensionando si es necesario.

#     Args:
#         tomo_root_path (str): Carpeta raíz donde están los tomos (carpetas tomo_*).
#         csv_path (str): Ruta al CSV con la info de los tomos.
#         num_images (int): Número de imágenes por tomo (default=32).
#         image_size (int): Tamaño esperado de la imagen (512).
#         num_tomos (int or None): Número máximo de tomos a cargar.

#     Returns:
#         tuple: (X, y)
#             - X: np.ndarray con shape (num_tomos, 32, 512, 512, 1)
#             - y: np.ndarray con etiquetas binarias Has_motors por tomo
#     """

#     df = pd.read_csv(csv_path)
#     tomo_dirs = sorted(glob(os.path.join(tomo_root_path, 'tomo_*')))
#     if num_tomos is not None:
#         tomo_dirs = tomo_dirs[:num_tomos]

#     all_tomos = []
#     all_labels = []

#     for tomo_dir in tqdm(tomo_dirs, desc="Cargando tomos"):
#         tomo_id = os.path.basename(tomo_dir)

#         # Obtener etiqueta binaria
#         row = df[df['tomo_id'] == tomo_id]
#         if row.empty:
#             print(f"⚠️ No se encontró info para {tomo_id}")
#             continue
#         num_motors = 1 if row['Number_of_motors'].values[0] > 0 else 0

#         # Cargar imágenes (png/jpg)
#         image_paths = sorted(glob(os.path.join(tomo_dir, "*.png")) + glob(os.path.join(tomo_dir, "*.jpg")))

#         if len(image_paths) < num_images:
#             print(f"⚠️ {tomo_id} tiene menos de {num_images} imágenes, se ignorará")
#             continue

#         # Leer y redimensionar las primeras num_images imágenes
#         tomo_imgs = []
#         for img_path in image_paths[:num_images]:
#             img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#             if img is None:
#                 print(f"⚠️ No se pudo leer imagen {img_path}")
#                 break

#             if img.shape != (image_size, image_size):
#                 img = cv2.resize(img, (image_size, image_size), interpolation=cv2.INTER_AREA)

#             tomo_imgs.append(img.astype(np.float32) / 255.0)  # Normalizar

#         if len(tomo_imgs) != num_images:
#             continue  # Si falla alguna imagen saltar tomo

#         # Convertir a numpy array y añadir canal
#         tomo_array = np.expand_dims(np.stack(tomo_imgs, axis=0), axis=-1)  # shape (32,512,512,1)
#         all_tomos.append(tomo_array)
#         all_labels.append(num_motors)

#     if not all_tomos:
#         print("⚠️ No se cargó ningún tomo correctamente.")
#         return None, None

#     X = np.stack(all_tomos, axis=0)  # shape (num_tomos, 32, 512, 512, 1)
#     y = np.array(all_labels)

#     return X, y


In [8]:
# X, Y = load_tomos_images_array (tomo_root_path= ,
#                                 csv_path= ,
#                                 num_images=32,
#                                 image_size=512,
#                                 num_tomos=None
#                                 )

In [9]:
print ( X.shape), print ( y.shape)

NameError: name 'X' is not defined

In [8]:

random_state=42

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.05,
                                                    random_state=random_state)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.2,
                                                  random_state=random_state)

In [ ]:

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow_addons.metrics import FBetaScore
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
# from tensorflow.keras.metrics import Recall, Precision


def init_model(X):
    original_input_shape = X.shape[1:]

    #  force input to 3 channel if it's grey (1 canal)
    if original_input_shape[-1] == 1:
        base_input_shape = original_input_shape[:-1] + (3,)
    else:
        base_input_shape = original_input_shape

    # load DenseNet121 without last classify layer
    base_model = DenseNet121(include_top=False,
                             weights='imagenet',
                             input_shape=base_input_shape
                             )
    base_model.trainable = False

    # Input layer
    inputs = Input(shape=original_input_shape)

    # convert in RGB if need it,
    if original_input_shape[-1] == 1:
        x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(inputs)
    else:
        x = inputs

    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-4),
        metrics=[
            'accuracy',
            FBetaScore(num_classes=1,
                       average='micro',
                       beta=2.0),
            Recall(),
            Precision(),
        ]
    )

    return model


def model_train (model, X_train_processed, y_train, batch_size=16 ,epochs=5):
    verbose=1
    validation_split=0.1
    es = EarlyStopping(patience=3,
                       restore_best_weights=True)

    history = model.fit(X_train_processed,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        # validation_split=validation_split,
                        validation_data=(X_val, y_val),
                        callbacks=[es],
                        verbose=verbose)
    return history


In [10]:
model = init_model(X_train)

In [ ]:

history = model_train(model, X_train,y_train)
history

In [ ]:
history.history


In [ ]:
def plot_training_metrics(history):
    """
   plot the metrics, even if there is not one of them into 'history'
    """
    history_dict = history.history
    plotted = False

    def safe_plot(metric_name, color, linestyle='-'):
        nonlocal plotted
        if metric_name in history_dict:
            label = metric_name.replace('_', ' ')
            plt.plot(history_dict[metric_name], c=color, linestyle=linestyle, label=label)
            plotted = True

    # fbeta
    safe_plot('fbeta_score', 'r')
    safe_plot('val_fbeta_score', 'r', '--')

    # recall
    safe_plot('recall', 'g')
    safe_plot('val_recall', 'g', '--')
    safe_plot('recall_1', 'g')  # alternativa por si usa nombre automático
    safe_plot('val_recall_1', 'g', '--')

    # accuracy
    safe_plot('accuracy', 'black')
    safe_plot('val_accuracy', 'black', '--')

    # precision
    safe_plot('precision', 'blue')
    safe_plot('val_precision', 'blue', '--')

    if plotted:
        plt.title("Training Metrics")
        plt.xlabel("Epoch")
        plt.ylabel("Metric Value")
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("⚠️ No se encontró ninguna métrica conocida para graficar.")


plot_training_metrics(history)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_pred= model.predict(X_test )
y_pred

In [ ]:
from sklearn.metrics import fbeta_score

# Obtener predicciones del modelo para el set de validación
y_val_pred_prob = model.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype(int)

# Calcular F-beta ponderado
# fbeta = fbeta_score(y_val, y_val_pred, beta=2, average='weighted')
fbeta = fbeta_score(y_val, y_val_pred, beta=2)
print(f"F-beta score ponderado (beta=2) en validación: {fbeta:.4f}")

In [ ]:
X_test.shape